In [1]:
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine
from sqlalchemy.pool import Pool
import mysql.connector
import json
import csv
import os

import numpy as np
import pymysql as mysql
import matplotlib.pyplot as plt
import requests
import getpass

!pip install mysql-connector-python

In [2]:
###############################################
# Cleaning the Data
###############################################

# Extract the Data 

In [97]:
#extract the data 


go_1k = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_1k.csv"))
go_daily_sales = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_daily_sales.csv"))
go_methods = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_methods.csv"))
go_products = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_products.csv"))
go_retailers = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/go_retailers.csv"))
cpi_monthly = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/1_month_percent_change_in_CPI.csv"))
unemploy_monthly = pd.DataFrame(pd.read_csv("https://github.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/raw/main/GOSales_data/Monthly_unemployment_rate.csv"))
cci_monthly = pd.DataFrame(pd.read_csv("https://raw.githubusercontent.com/RenettaNelson/ADS-507---SQL-Data-Pipeline/main/GOSales_data/CCI%20CSV.csv"))

In [69]:
go_1k.shape

(1000, 6)

In [4]:
go_1k['Retailer code'].describe

<bound method NDFrame.describe of 0      1115
1      1115
2      1115
3      1132
4      1132
       ... 
995    1749
996    1756
997    1760
998    1762
999    1766
Name: Retailer code, Length: 1000, dtype: int64>

In [5]:
go_1k.head()

,Retailer code,Product number,Date,Quantity
0,1115,125110,2016-02-09,46
1,1115,144180,2016-04-21,19
2,1115,149140,2017-02-14,11
3,1132,92110,2017-06-13,232
4,1132,96110,2018-06-20,37


In [6]:
go_daily_sales.head()

,Retailer code,Product number,Order method code,Date,Quantity,Unit price,Unit sale price
0,1201,109110,4,2015-01-12,648,76.86,71.48
1,1201,112110,4,2015-01-12,799,10.64,10.21
2,1201,115110,4,2015-01-12,755,10.71,10.28
3,1205,70240,3,2015-01-12,70,122.70,114.11
4,1205,71110,3,2015-01-12,28,95.62,92.75


In [7]:
cci_monthly.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-01,98.33976,NaN
1,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-02,98.64955,NaN
2,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-03,98.96025,NaN
3,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-04,99.13481,NaN
4,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-05,99.21778,NaN


# Data wrangling

In [8]:
cci_monthly.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-01,98.33976,NaN
1,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-02,98.64955,NaN
2,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-03,98.96025,NaN
3,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-04,99.13481,NaN
4,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-05,99.21778,NaN


In [98]:
cci_monthly['Year'] = pd.DatetimeIndex(cci_monthly['TIME']).year
cci_monthly['Month'] = pd.DatetimeIndex(cci_monthly['TIME']).month
cci_monthly.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes,Year,Month
0,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-01,98.33976,NaN,2014,1
1,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-02,98.64955,NaN,2014,2
2,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-03,98.96025,NaN,2014,3
3,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-04,99.13481,NaN,2014,4
4,G-20,CCI,AMPLITUD,LTRENDIDX,M,2014-05,99.21778,NaN,2014,5


In [ ]:
drop

In [99]:
go_daily_sales['Year'] = pd.DatetimeIndex(go_daily_sales['Date']).year
go_daily_sales['Month'] = pd.DatetimeIndex(go_daily_sales['Date']).month
go_daily_sales.head()

,Retailer code,Product number,Order method code,Date,Quantity,Unit price,Unit sale price,Year,Month
0,1201,109110,4,2015-01-12,648,76.86,71.48,2015,1
1,1201,112110,4,2015-01-12,799,10.64,10.21,2015,1
2,1201,115110,4,2015-01-12,755,10.71,10.28,2015,1
3,1205,70240,3,2015-01-12,70,122.70,114.11,2015,1
4,1205,71110,3,2015-01-12,28,95.62,92.75,2015,1


In [100]:
go_1k['Year'] = pd.DatetimeIndex(go_1k['Date']).year
go_1k['Month'] = pd.DatetimeIndex(go_1k['Date']).month
go_1k.head()

,Retailer code,Product number,Date,Quantity,Year,Month
0,1115,125110,2016-02-09,46,2016,2
1,1115,144180,2016-04-21,19,2016,4
2,1115,149140,2017-02-14,11,2017,2
3,1132,92110,2017-06-13,232,2017,6
4,1132,96110,2018-06-20,37,2018,6


In [101]:
go_1k.columns = go_1k.columns.str.replace(' ', '_')
go_1k.head()

,Retailer_code,Product_number,Date,Quantity,Year,Month
0,1115,125110,2016-02-09,46,2016,2
1,1115,144180,2016-04-21,19,2016,4
2,1115,149140,2017-02-14,11,2017,2
3,1132,92110,2017-06-13,232,2017,6
4,1132,96110,2018-06-20,37,2018,6


In [102]:
go_daily_sales.columns = go_daily_sales.columns.str.replace(' ', '_')
go_daily_sales.head()

,Retailer_code,Product_number,Order_method_code,Date,Quantity,Unit_price,Unit_sale_price,Year,Month
0,1201,109110,4,2015-01-12,648,76.86,71.48,2015,1
1,1201,112110,4,2015-01-12,799,10.64,10.21,2015,1
2,1201,115110,4,2015-01-12,755,10.71,10.28,2015,1
3,1205,70240,3,2015-01-12,70,122.70,114.11,2015,1
4,1205,71110,3,2015-01-12,28,95.62,92.75,2015,1


In [131]:
# go_1k['Date'] = go_1k['Date'].values.astype('datetime64[M]')
# print (go_1k)

In [132]:
# go_daily_sales['Date'] = go_daily_sales['Date'].values.astype('datetime64[M]')
# print (go_daily_sales)



In [133]:
# ### Correct time to match other tables format. 
# cci_monthly['TIME'] = cci_monthly['TIME'].values.astype('datetime64[M]')
# print (cci_monthly)

In [104]:
cci_monthly = cci_monthly.rename(columns={"TIME": "Date"})

In [105]:
### drop flag codes columns due to NAN and irrelevance
cci_monthly = cci_monthly.drop(['Flag Codes', 'LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Date'], axis=1)

In [106]:
go_1k = go_1k.drop(['Date'], axis=1)

In [135]:
# go_1k = go_1k.drop(['Retailer_code'], axis=1)

In [136]:
# go_daily_sales = go_daily_sales.drop(['Retailer_code'], axis=1)

# Create Tables & Insert Records

In [85]:
###############################################
# Creating the Tables
###############################################

In [110]:
test = pd.merge(cci_monthly, go_1k, how='inner',
                 left_on=['Month', 'Year'],
                 right_on=['Month', 'Year'])
merged.head()

,Value,Year,Month,Retailer_code,Product_number,Quantity
0,99.97893,2015,1,1133,134120,222
1,99.97893,2015,1,1137,25110,198
2,99.97893,2015,1,1137,144180,21
3,99.97893,2015,1,1192,143120,159
4,99.97893,2015,1,1194,125120,16


In [111]:
test.head()

,Value,Year,Month,Retailer_code,Product_number,Quantity
0,99.97893,2015,1,1133,134120,222
1,99.97893,2015,1,1137,25110,198
2,99.97893,2015,1,1137,144180,21
3,99.97893,2015,1,1192,143120,159
4,99.97893,2015,1,1194,125120,16


In [112]:
test.shape

(1000, 6)

In [28]:
# test = pd.concat([cci_monthly, go_1k, go_daily_sales])

In [113]:
test.isnull().sum()

Value             0
Year              0
Month             0
Retailer_code     0
Product_number    0
Quantity          0
dtype: int64

In [114]:
test.isnull().values.any()

False

In [115]:
test.dtypes

Value             float64
Year                int64
Month               int64
Retailer_code       int64
Product_number      int64
Quantity            int64
dtype: object

In [116]:
test.dtypes

Value             float64
Year                int64
Month               int64
Retailer_code       int64
Product_number      int64
Quantity            int64
dtype: object

In [117]:
test.head()

,Value,Year,Month,Retailer_code,Product_number,Quantity
0,99.97893,2015,1,1133,134120,222
1,99.97893,2015,1,1137,25110,198
2,99.97893,2015,1,1137,144180,21
3,99.97893,2015,1,1192,143120,159
4,99.97893,2015,1,1194,125120,16


In [25]:
# # Connect to the sql server

# import mysql.connector as mysql
# from mysql.connector import Error

# try:
#     conn = mysql.connect(host='localhost',
#                          user='root',
#                          password=getpass.getpass(prompt='Password:',stream=None))
#     if conn.is_connected():
#         cursor = conn.cursor()
#         cursor.execute("CREATE DATABASE go_1kDB")
#         print("go1k_DB database created")
        
# except Error as e:
#     print("Error connecting to MySQL", e)

Password:········
testDB database created


In [119]:
# Connect to the sql server

import mysql.connector as mysql
from mysql.connector import Error

try:
    conn = mysql.connect(host='localhost',
                         user='root',
                         password=getpass.getpass(prompt='Password:',stream=None))
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE testDB")
        print("test_DB database created")
        
except Error as e:
    print("Error connecting to MySQL", e)

Password:········
test_DB database created


In [32]:
# try:
#     conn = mysql.connect(host='localhost',
#                         database='go_1kDB', user='root',
#                         password=getpass.getpass(prompt='Password:',stream=None))
#     if conn.is_connected():
#         cursor = conn.cursor()
#         cursor.execute("select database();")
#         record = cursor.fetchone()
#         print("You are connected to the db: ", record)
#         cursor.execute('DROP TABLE IF EXISTS go_1k;')
#         print('Creating table....')
#         cursor.execute("CREATE TABLE go_1k (Retailer_code INT(255)NOT NULL, Product_number INT(255)Not NULL, Date CHAR(20)NOT NULL,Quantity INT(255)NOT NULL, Year INT(255) NOT NULL, Month INT(255)NOT NULL)")
#         print("go_1k table is created....")
#         for i, row in go_1k.iterrows():
#             sql = "INSERT INTO go_1kDB.go_1k VALUES (%s,%s,%s,%s,%s,%s)"
#             cursor.execute(sql, tuple(row))
#             print("Record Inserted")
#                        # the connection is not autocommited by default, so we must commit
#                        # to save our changes
#             conn.commit()
# except Error as e:
#                        print("Error while connected to MySQL", e)

In [121]:
test.dtypes

Value             float64
Year                int64
Month               int64
Retailer_code       int64
Product_number      int64
Quantity            int64
dtype: object

In [122]:
try:
    conn = mysql.connect(host='localhost',
                        database='testDB', user='root',
                        password=getpass.getpass(prompt='Password:',stream=None))
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You are connected to the db: ", record)
        cursor.execute('DROP TABLE IF EXISTS test;')
        print('Creating table....')
        cursor.execute("CREATE TABLE test (Retailer_code INT(255)NOT NULL, Product_number INT(255)NOT NULL, Quantity INT(255)NOT NULL, Year INT(255) NOT NULL, Month INT(255)NOT NULL, Value FLOAT(20)NOT NULL)")
        print("test table is created....")
        for i, row in test.iterrows():
            sql = "INSERT INTO testDB.test VALUES (%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record Inserted")
                       # the connection is not autocommited by default, so we must commit
                       # to save our changes
            conn.commit()
except Error as e:
                       print("Error while connected to MySQL", e)

Password:········
You are connected to the db:  ('testdb',)
Creating table....
test table is created....
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted

Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record Inserted
Record I

In [123]:
# execute query
sql = "SELECT * FROM test"
cursor.execute(sql)

#fetch all records
result = cursor.fetchall()
for i in result:
    print(i)

(100, 2015, 1, 1133, 134120, 222.0)
(100, 2015, 1, 1137, 25110, 198.0)
(100, 2015, 1, 1137, 144180, 21.0)
(100, 2015, 1, 1192, 143120, 159.0)
(100, 2015, 1, 1194, 125120, 16.0)
(100, 2015, 1, 1201, 22110, 333.0)
(100, 2015, 1, 1206, 144150, 12.0)
(100, 2015, 1, 1218, 124190, 37.0)
(100, 2015, 1, 1223, 145170, 2.0)
(100, 2015, 1, 1268, 147180, 17.0)
(100, 2015, 1, 1271, 147110, 24.0)
(100, 2015, 1, 1272, 99110, 445.0)
(100, 2015, 1, 1272, 126130, 42.0)
(100, 2015, 1, 1272, 143120, 17.0)
(100, 2015, 1, 1277, 125110, 98.0)
(100, 2015, 1, 1282, 12110, 77.0)
(100, 2015, 1, 1282, 145170, 16.0)
(100, 2015, 1, 1405, 128140, 76.0)
(100, 2015, 2, 1137, 23110, 265.0)
(100, 2015, 2, 1148, 23110, 265.0)
(100, 2015, 2, 1192, 143110, 95.0)
(100, 2015, 2, 1196, 141110, 42.0)
(100, 2015, 2, 1196, 143110, 4.0)
(100, 2015, 2, 1229, 106110, 38.0)
(100, 2015, 2, 1235, 97110, 313.0)
(100, 2015, 2, 1238, 147180, 200.0)
(100, 2015, 2, 1247, 126130, 271.0)
(100, 2015, 2, 1274, 102110, 87.0)
(100, 2015, 2, 1311

In [114]:
if (conn.is_connected()):
    cursor.close()
    conn.close()
    print("MySQL connection is closed")

MySQL connection is closed


In [ ]:
# import the module
from sqlalchemy import create_engine
#create sqlalchemy engine


In [87]:
# # Create Tables
# query=["CREATE TABLE go_1k (log_date VARCHAR(200), sensor_id VARCHAR(100), temperature VARCHAR(100), vibration VARCHAR(100))",
#            "CREATE TABLE go_daily_sales (equipment_id VARCHAR(100), code VARCHAR(100), group_name VARCHAR(100))",
#            "CREATE TABLE equipment_sensors (equipment_id VARCHAR(100), sensor_id VARCHAR(100))",
#            "CREATE TABLE equipment_sensors (equipment_id VARCHAR(100), sensor_id VARCHAR(100))",

#                "CREATE TABLE go_methods (Year, Jan, Feb, March, April, May, June, July, August, September, October, November, December , sensor_id VARCHAR(100))",

#                    "CREATE TABLE go_products (equipment_id VARCHAR(100), sensor_id VARCHAR(100))",

#                        "CREATE TABLE go_retailers (equipment_id VARCHAR(100), sensor_id VARCHAR(100))",
                
#                 "CREATE TABLE cpi_monthly (equipment_id VARCHAR(100), sensor_id VARCHAR(100))",
                
#                 "CREATE TABLE unemploy_monthly(equipment_id VARCHAR(100), sensor_id VARCHAR(100))",
                
#                 "CREATE TABLE cci_monthly (equipment_id VARCHAR(100), sensor_id VARCHAR(100))"]

#     for i in query:
#         db_cursor.execute(i)
#         db_connection.commit()

In [127]:
#create engine to add data to sql database "Wild_and_Wonderful_Sales" and import records
engine_data = 'mysql+mysqlconnector://' + 'root' + ':' + getpass.getpass(prompt='Password:',stream=None) + '@' + 'localhost' + ':3306/' \
       + 'go_1kDB' + '?charset=utf8mb4'
engine = create_engine(engine_data)

# Insert whole DataFrame into MySQL
go_1k.to_sql('go_1k', con = engine, if_exists = 'append', chunksize = 1000, index=False)

Password:········


-1

In [129]:
# Execute Query
sql = "SELECT * FROM go_1k"
cursor.execute(sql)

# Fetch all records
result = cursor.fetchall()
for i in result:
    print(i)

(1115, 125110, '2016-02-09', 46, 2016, 2)
(1115, 144180, '2016-04-21', 19, 2016, 4)
(1115, 149140, '2017-02-14', 11, 2017, 2)
(1132, 92110, '2017-06-13', 232, 2017, 6)
(1132, 96110, '2018-06-20', 37, 2018, 6)
(1133, 123140, '2017-06-12', 143, 2017, 6)
(1133, 125150, '2017-02-15', 48, 2017, 2)
(1133, 125150, '2018-05-18', 23, 2018, 5)
(1133, 126140, '2017-06-19', 24, 2017, 6)
(1133, 128140, '2017-03-14', 152, 2017, 3)
(1133, 128140, '2017-09-13', 61, 2017, 9)
(1133, 129180, '2017-08-18', 4, 2017, 8)
(1133, 134120, '2015-01-23', 222, 2015, 1)
(1133, 135120, '2018-05-18', 11, 2018, 5)
(1133, 136140, '2017-10-17', 19, 2017, 10)
(1133, 143110, '2015-11-10', 95, 2015, 11)
(1133, 144180, '2016-04-27', 9, 2016, 4)
(1134, 63140, '2016-07-19', 57, 2016, 7)
(1134, 68110, '2017-08-09', 22, 2017, 8)
(1134, 70240, '2017-07-11', 35, 2017, 7)
(1134, 144180, '2017-05-16', 43, 2017, 5)
(1135, 45110, '2018-03-12', 71, 2018, 3)
(1135, 126140, '2016-07-12', 32, 2016, 7)
(1135, 127110, '2015-06-16', 45, 201

In [89]:
# Execute the to_sql for writting DF into SQL and add to database
go_1k.to_sql('go_1k', engine, if_exists='append', index=False) 
go_daily_sales.to_sql('go_daily_sales', engine, if_exists='append', index=False) 
go_methods.to_sql('go_methods', engine, if_exists='append', index=False) 
go_products.to_sql('go_products', engine, if_exists='append', index=False) 
go_retailers.to_sql('go_retailers', engine, if_exists='append', index=False) 
cpi_monthly.to_sql('cpi_monthly', engine, if_exists = 'append', index=False)
unemploy_monthly.to_sql('unemploy_monthly', engine, if_exists = 'append', index=False)
cci_monthly.to_sql('cci_monthly', engine, if_exists = 'append', index=False)

-1

In [90]:
#update connection to use Wild and Wonderful Sales database
connection = mysql.connect(host='localhost',
                         user='root',
                         password=getpass.getpass(prompt='Password:',stream=None),
                         database='Wild_and_Wonderful_Sales')

Password:········


In [91]:
################################################################
# RETRIEVING THE RECORDS
################################################################
        

In [92]:
connection.close()